# Embed holoviews interactive charts in a folium map (0.0.1-alpha)

The output of this notebook can be seen at:

https://sfsheath.github.io/holoviews-in-folium/embedded_iframe.html.

The data comes from https://github.com/sfsheath/roman-amphitheaters.

In [ ]:
import folium
import geopandas as gpd
import io
import json
import numpy as np
import pandas as pd
import requests
import holoviews as hv
hv.extension('bokeh')

from bokeh.models import HoverTool
from folium import FeatureGroup, LayerControl, Map, Marker
from IPython.display import HTML,IFrame

import matplotlib
import matplotlib.cm as cm

%matplotlib inline

## Load Data

In [ ]:
ramphs_df = pd.read_csv('http://sebastianheath.com/roman-amphitheaters/roman-amphitheaters.csv')

In [ ]:
# set up df for an index plot
extmajors_df = ramphs_df.sort_values(by = 'extmajor', ascending=False).reset_index()[['index','label','extmajor','latitude','longitude']]

extmajors_df['index'] = extmajors_df.index

extmajors_df.index = extmajors_df.label

extmajors_df.head(2)

## Simple Case

In [ ]:
# create an index plot
hover = HoverTool(tooltips=[("", "<b>@label</b>")])
index_plot = hv.Points(extmajors_df[['index', 'extmajor','label']].dropna())\
 .opts(plot=dict(height=300, width=400, tools=[hover])).relabel("Index Plot of Roman Amphitheaters")

In [ ]:
# get lat/long of Colosseum. That's as good as any center point for map
latitude = extmajors_df.loc['Colosseum','latitude']
longitude = extmajors_df.loc['Colosseum','longitude']

# create map
m = folium.Map((latitude, longitude), zoom_start=4)

# renderer.save() will 'write' to 'f', which can then be 'read'.
renderer = hv.renderer("bokeh")
f = io.BytesIO()
renderer.save(index_plot,f)
html = f.read().decode("utf-8")
f.close() # I think this is good practice
# the variable html can now be displayed in an iframe
    
# create an iframe using the html variable, then a popup with that iframe as content
iframe = folium.IFrame(html=html, width=450, height=300)
popup = folium.Popup(iframe, max_width=2650)

# create a marker with the popup
folium.Marker([latitude, longitude], popup=popup).add_to(m)

#display the map
m

In [ ]:
html

## More Ambitious: Index Plots with Highlight for Every Amphitheater
Or at least those amphitheaters with known exterior lengths. And a bunch o' layers FTW!

In [ ]:
# choose which amphitheaters to highlight
# use_labels = ramphs_df.query("label in ['Colosseum','Arles','Mactaris','Bern','Cassino']").label.tolist() 

# this will plot all amphitheaters with known exterior lengths
use_labels = ramphs_df[ramphs_df.extmajor.notna()].sort_values(by = 'extmajor', ascending = True).label.tolist()

In [ ]:
# An index plot of all known max exterior lengths that is the 'background'
# for the individual plots that highlight individual amphitheater length
hover = HoverTool(tooltips=[("", "<b>@label</b>")])

index_plot = hv.Points(extmajors_df[['index', 'extmajor','label']].dropna())\
 .opts(plot=dict(height=300, width=400, tools=[hover]))

In [ ]:
# create a color mapper

max_value = extmajors_df.extmajor.max()
min_value = extmajors_df.extmajor.min()

norm = matplotlib.colors.Normalize(vmin=min_value-10, vmax=max_value, clip=False) # subtracting from min is simple way to intensify color
mapper = cm.ScalarMappable(norm=norm, cmap=cm.YlOrRd)


In [ ]:
# loop through 'use_labels' to make a horizontal line showing exterior length of that amphitheater   
maps = {} # this will be a python dictionary that holds all holoviews layouts
for l in use_labels:
    
    r,g,b,a = mapper.to_rgba(extmajors_df.loc[l,'extmajor'])
    color = "#{:02x}{:02x}{:02x}".format(int(r*255),int(g*255),int(b*255))

    h_line = hv.HLine(extmajors_df.loc[l,'extmajor']).options(color = color, line_width= 2, alpha=0.5 )
    
    # 'multiply' background and horizontal line for each amphtitheater, add result to dictionary
    maps[l] = (index_plot * h_line).relabel("Exterior lengths with {} highlighted".format(l))

## Make Folium map
The content of each iframe is the holoviews viz as captured in a BytesIO variable

In [ ]:
# This cell makes a provinces_g folium.FeatureGroup that can be added later

# grab the geojson from the "Digital Atlas of the Roman Empire", convert to geodataframe
provinces_geojson = "http://cdn.rawgit.com/klokantech/roman-empire/ece9d9a65e09a5596bafe4a8bf9b288c3c511907/data/provinces.geojson"
r = requests.get(provinces_geojson)
data = r.json()
provinces_gdf = gpd.GeoDataFrame.from_features(data['features'])
provinces_gdf.crs = {'init' :'epsg:4326'}

style_function = lambda x: {'fillColor': '#bdb','color':'#bbd', 'weight': .5}

provinces_g = folium.FeatureGroup(name = "Roman Provinces (200 CE)")

for index,row in provinces_gdf.iterrows():
    # overly verbose as the folium.GeoJson method has no 'popup' parameter
    prov_tmp_gdf = provinces_gdf.query("name == '{}'".format(row['name']))
    prov_feature = folium.GeoJson(prov_tmp_gdf, style_function = style_function).add_to(provinces_g)
    popup = folium.Popup("<b>{}</b>".format(row['name']))
    popup.add_to(prov_feature)

In [ ]:
renderer = hv.renderer("bokeh")

attr = '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> &copy; <a href="http://cartodb.com/attributions">CartoDB</a>',
tiles = "https://cartodb-basemaps-a.global.ssl.fastly.net/light_nolabels/{z}/{x}/{y}.png"

m = folium.Map((42, 13),
               zoom_start=4)


folium.TileLayer(attr=attr,tiles=tiles,name="Carto Gray").add_to(m)

# add the previously created provinces
provinces_g.add_to(m)


#  create a group for amphitheaters with no data
# add this first so it is behind next group
nodata_g = folium.FeatureGroup(name = "Size not Recorded (yet?)")

for index,row in ramphs_df[ramphs_df.extmajor.isna()].iterrows():
    latitude, longitude = row[['latitude','longitude']]
    
    folium.CircleMarker([latitude, longitude],
                        radius=2,popup='<b>{}</>'.format(row.label),
                        color = '#999', fill=True, fill_color = '#999').add_to(nodata_g)
nodata_g.add_to(m)


# create a group to hold amphitheaters with hv popups
extmajors_g = folium.FeatureGroup(name = "Known Size")

for l in use_labels:
    # get the lat/long of the amphitheater
    latitude, longitude = extmajors_df.loc[l,['latitude','longitude']]

    # renderer.save() will 'write' to 'f', which can then be 'read'.
    f = io.BytesIO()
    renderer.save(maps[l],f)
    html = f.read().decode("utf-8")
    f.close() # I think this is good practice
    
    # create an iframe using the html variable, then a popup with that iframe as content
    iframe = folium.IFrame(html=html, width=450, height=300)
    popup = folium.Popup(iframe, max_width=2650)

    # create a color based on extmajor
    r,g,b,a = mapper.to_rgba(extmajors_df.loc[l,'extmajor'])
    color = "#{:02x}{:02x}{:02x}".format(int(r*255),int(g*255),int(b*255))
    
    # create a marker
    folium.CircleMarker([latitude, longitude],
                        radius=3,popup=popup,
                        color = color, fill=True, fill_color = color).add_to(extmajors_g)
extmajors_g.add_to(m)

folium.LayerControl().add_to(m)

m.save("embedded_iframe.html")
m